# Function Calling with LangChain Agents

In this notebook, we will leverage [Agents](https://python.langchain.com/docs/modules/agents/) in LangChain (particularly, custom-defined agents) to demonstrate the capabilities of function-calling-augmented LLMs. 

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_name="gpt-3.5-turbo"
)

## 1. Calculator Function

Computers can solve complex math problems, yet if we ask GPT-3.5 (or GPT-4, for that matter) for the answer to `(4.5*2.1)^2.2`, it fails.

In [ ]:
print(llm.invoke("(4.5*2.1)^2.2").content)

To address this problem, we can leverage function calling to allow the LLM to use an actual calculator tool.

### 1.1 Creating a Custom Tool

We will create a tool that will evaluate basic arithmetic math problems. There many ways to define tools - we enumerate some of these approaches below.

#### 1.1.1 OOP Approach

We can define our own tools with an inheritance-based approach. 

In [ ]:
from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    # The name and description are required by LangChain - they are used to help the LLM figure out what tool to use
    name = "Calculator"
    description = "A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."
    
    def _run(self, input_expr: str) -> int:
        # Evaluate the mathematical expression to get the result
        # Replace ^ (mathematical exponent operator) with ** (python exponent operator)
        return eval(input_expr.replace("^", "**"), {}, {})

Note that the `eval` function is generally very dangerous (but for the sake of this example, it's fine). `eval(str)` evaluates the input string as a literal Python expression, running whatever is contained inside the string. For instance, `eval("4.5 * 2")` returns `9`, and `eval("[1, 2, 3]")` returns a Python list with the values 1, 2, and 3. However, what this means is that `eval("os.system("rm -rf /")")` can also be ran...

The two empty dictionaries we also pass as argument help to prevent some attacks, such as the `os.system` one; refer to the `eval` function's [documentation](https://realpython.com/python-eval-function/) to learn why. In practice, you would be best served using the `literal_eval` function from the `ast` library.

#### 1.1.2 Function Decorator Approach

Another method for custom tool definition is through the use of function [decorators](https://www.geeksforgeeks.org/decorators-in-python/) (a more advanced Python concept).

In [ ]:
from langchain.tools import tool

@tool
def calculator(input_expr: str) -> int:
    """A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."""
    return eval(input_expr.replace("^", "**"), {}, {})

#### Defining an Agent

In [ ]:
# when giving tools to LLM, we must pass them as a list of tools
# tools = [CalculatorTool]
tools = [calculator]

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")

Note: If you are running into weird errors with the tools, try restarting your kernel.

### 1.2 Using an Existing Tool

LangChain's [LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html) would be a good solution here. The LLMMathChain turns math problems into Python Code (using the LLM) and runs the code to obtain the result. This would be very helpful for more complicated math problems.

In [ ]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass them as a list of tools
tools = [math_tool]

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")

## 2. Your Turn

Now, create a function calling application of your own. You are encouraged to use external APIs to create robust tools. Refer to `simple-weather-llm.py` for an example tool.

In [ ]:
class MyTool(BaseTool):
    name = "The name of the tool"
    description = "A description of the tool"

    # change the arguments as needed
    def _run(self, **kwargs):
        return "Hello, World!"